#Loading Data and libraries

In [ ]:
!pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 5.4 MB/s 
     |████████████████████████████████| 462 kB 45.2 MB/s 


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import re
from sklearn.metrics import f1_score
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Input, Dropout, LSTM
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.metrics import f1_score, confusion_matrix
from keras.models import load_model
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/kaggle_toxic/

/content/drive/MyDrive/kaggle_toxic


In [ ]:
def loading_things():
    global bert_preprocess, bert
    bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
    bert = load_model('bert.hdf5', custom_objects={'KerasLayer': bert_preprocess})

In [ ]:
loading_things()

In [ ]:
all_data = pd.read_csv('all_data.csv')

In [ ]:
toxic = []
#making comments which have probability more than 0.5 as toxic and marking them as 1 while non-toxic as 0
for i in all_data['toxicity']:
     if i > 0.5:
        toxic.append(1)
     else:
        toxic.append(0)

all_data['toxic_binary'] = toxic

In [ ]:
all_data['sub_toxic'] = all_data[['severe_toxicity','obscene','sexual_explicit', 'identity_attack','insult', 'threat']].idxmax(axis=1)

In [ ]:
sub_toxic = []
for j in range(len(all_data)):
   if all_data['toxic_binary'][j] == 1:
      if all_data['sub_toxic'][j] == 'severe_toxicity':
          sub_toxic.append(6)  
      if all_data['sub_toxic'][j] == 'obscene':  
          sub_toxic.append(5)
      if all_data['sub_toxic'][j] == 'sexual_explicit': 
          sub_toxic.append(4)
      if all_data['sub_toxic'][j] == 'identity_attack':     
          sub_toxic.append(3)
      if all_data['sub_toxic'][j] == 'insult': 
          sub_toxic.append(2)
      if all_data['sub_toxic'][j] == 'threat':   
          sub_toxic.append(1)
   if all_data['toxic_binary'][j] == 0:
       sub_toxic.append(0)

all_data['sub_toxic'] = sub_toxic

#Helper functions

In [ ]:
def clean(text):
  text_token = word_tokenize(text)
  filtered_text = ' '.join([w.lower() for w in text_token if len(w) > 2])
  filtered_text = filtered_text.replace(r"[^a-zA-Z]+", '')
  text_only = re.sub(r'\b\d+\b', '', filtered_text)
  clean_text = text_only.replace(',', '').replace('.', '').replace(':', '')
  return clean_text

In [ ]:
def printing_function(text, category):
    result = {}
    for i in range(len(category)):
        if category[i] == 0:
           result['Sentence-'+str(i)] = 'Classification : non-toxic'
        if category[i] == 1:
           result['Sentence-'+str(i)] = 'Classification : threat'
        if category[i] == 2:
           result['Sentence-'+str(i)] = 'Classification : insult'
        if category[i] == 3:
           result['Sentence-'+str(i)] = 'Classification : identity attack'
        if category[i] == 4:
           result['Sentence-'+str(i)] = 'Classification : sexual explicit'
        if category[i] == 5:
           result['Sentence-'+str(i)] = 'Classification : obscene'
        if category[i] == 6:
           result['Sentence-'+str(i)] = 'Classification : severe toxicity'
    return result

#Predict function

In [ ]:
z = all_data['comment_text'][19]

In [ ]:
def predict_function(text):
    if type(text) == str:
       clean_text = [clean(str(text))]
    else:
       clean_text = [clean(str(x)) for x in text]
    prediction = bert.predict(clean_text)
    category = np.argmax(prediction,axis=1)
    p = printing_function(text, category)
    print(p)

In [ ]:
predict_function(z)

{'Sentence-0': 'Classification : obscene'}


In [ ]:
def predict_function_with_metric(text, true_class):
      if type(text) == str:
         clean_text = [clean(str(text))]
      else:
         clean_text = [clean(str(x)) for x in text]
      prediction = bert.predict(clean_text)
      category = np.argmax(prediction,axis=1)
      p = printing_function(text, category)
      print(p)
      if len(category) > 1:
         print('F1 score: ', f1_score(true_class, category, average=None))
         metric = {}
         for j in range(len(true_class)):
            if category[j] == true_class[j]:
               metric['Sentence-'+str(j)] = 'Correct prediction'
            
            if category[j] != true_class[j]:
               metric['Sentence-'+str(j)] = 'Wrong prediction'
         print(metric)
      else:
         if category == true_class:
            print('Correct prediction')
         if category != true_class:
            print('Wrong prediction')

In [ ]:
predict_function_with_metric(z, all_data['sub_toxic'][19])

{'Sentence-0': 'Classification : obscene'}
Correct prediction
